In [208]:
import Base: split

function split(array::Vector{T}, separator::T)::Vector{Vector{T}} where T
    chunks = Vector{T}[]
    current = T[]
    for elem in array
        if elem == separator
            push!(chunks, current)
            current = T[]
        else
            push!(current, elem)
        end
    end
    push!(chunks, current)
    return chunks
end

split (generic function with 5 methods)

In [219]:
polymers, instructions_raw = readlines("input.txt") |> arr -> split(arr, "")

2-element Vector{Vector{String}}:
 ["CHBBKPHCPHPOKNSNCOVB"]
 ["SP -> K", "BB -> H", "BH -> S", "BS -> H", "PN -> P", "OB -> S", "ON -> C", "HK -> K", "BN -> V", "OH -> F"  …  "VK -> H", "HO -> V", "CP -> F", "SF -> N", "FC -> P", "NO -> K", "VH -> S", "FN -> F", "PV -> O", "SC -> N"]

In [220]:
polymer = polymers[1]
instructions = split.(instructions_raw, " -> ") .|> Tuple .|> (t -> (t[1], only(t[2]))) |> Dict

Dict{SubString{String}, Char} with 100 entries:
  "FN" => 'F'
  "PO" => 'B'
  "BC" => 'K'
  "KF" => 'K'
  "CC" => 'F'
  "BV" => 'V'
  "OP" => 'V'
  "PF" => 'H'
  "BB" => 'H'
  "VN" => 'H'
  "NH" => 'K'
  "PP" => 'S'
  "NB" => 'V'
  "SF" => 'N'
  "CS" => 'S'
  "PN" => 'P'
  "CF" => 'F'
  "NO" => 'K'
  "KV" => 'V'
  "OS" => 'S'
  "VH" => 'S'
  "PS" => 'B'
  "BS" => 'H'
  "NP" => 'P'
  "HO" => 'V'
  ⋮    => ⋮

In [221]:
function do_step(polymer, instructions)
    insertions = map(i -> instructions[polymer[i:i+1]], 1:length(polymer)-1)
    zipped = zip(polymer, insertions)
    alterned = reduce((acc, elem) -> push!(acc, elem[1], elem[2]), zipped, init=Char[])
    push!(alterned, polymer[end]) # last element was missing
    return String(alterned)
end

function execute_steps(n, polymer, instructions)
    res = polymer
    for _ in 1:n
       res = do_step(res, instructions)
    end
    return res
end

execute_steps (generic function with 1 method)

In [222]:
using Pkg
Pkg.add("StatsBase")

import StatsBase: countmap

function part1(n, polymer, instructions)
    res = execute_steps(n, polymer, instructions)
    counts = (values ∘ countmap)(res)
    return maximum(counts) - minimum(counts)
end

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


part1 (generic function with 2 methods)

In [223]:
part1(10, polymer, instructions)

3118

In [228]:
reduce_pairs(l) = reduce((dico, pair) -> setindex!(dico, get(dico, pair[1], 0) + pair[2], pair[1]), l, init=Dict{typeof(l[1][1]), Int}())

function dictify(polymer)
    pairs = map(i -> polymer[i:i+1] => 1, 1:length(polymer)-1)
    return reduce_pairs(pairs)
end

function expanse_polymer_dict(key, polymer_dict, instructions)
    new_elem = instructions[key]
    new_key_1 = String([key[1], new_elem])
    new_key_2 = String([new_elem, key[2]])
    count = polymer_dict[key]
    return [(new_key_1, count), (new_key_2, count)]
end

function do_step_fast(polymer_dict, instructions)
    nested_counts = [expanse_polymer_dict(k, polymer_dict, instructions) for k in keys(polymer_dict)]
    new_pairs = vcat(nested_counts...)
    no_duplicates_pairs = reduce_pairs(new_pairs)
    return no_duplicates_pairs
end

function execute_steps_fast(n, polymer_dict, instructions)
    res = polymer_dict
    for _ in 1:n
       res = do_step_fast(res, instructions)
    end
    return res
end

function count_doubled_letters(polymer_dict, first_letter, last_letter)
    nested_counts = [[(k[1], polymer_dict[k]), (k[2], polymer_dict[k])] for k in keys(polymer_dict)]
    new_pairs = vcat(nested_counts...)
    counts = reduce_pairs(new_pairs)
    counts[first_letter] = get(counts, first_letter, 0) + 1
    counts[last_letter] = get(counts, last_letter, 0) + 1
    return counts
end

function part2(n, polymer, instructions)
    polymer_dict = dictify(polymer)
    res = execute_steps_fast(n, polymer_dict, instructions)
    counts = (values ∘ count_doubled_letters)(res, polymer[1], polymer[end])
    return (maximum(counts) - minimum(counts)) / 2
end

part2 (generic function with 1 method)

In [229]:
part2(40, polymer, instructions)

4.332887448171e12